In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
from skimage.metrics import structural_similarity as ssim
from math import log10, sqrt
import cpbd

def calculate_psnr(image1, image2):
    r"""Compute Peak Signal-to-Noise Ratio
    Args:
        image1          :   numpy.ndarray([H, W, C], dtype=np.float32)
        image2          :   numpy.ndarray([H, W, C], dtype=np.float32)
    Returns:
        PSNR            :   float (↑)
    """
    mse = np.mean((image1 - image2) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

total_ssim = []
total_psnr = []
total_cpbd = []
for i in range(6):
    ssim_emo = 0
    psnr_emo = 0
    cpbd_emo = 0
    img_list = sorted(glob(os.path.join(f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}', '*')))

    for pred_file in tqdm(img_list):
        gt_file = pred_file.replace(f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}',
                                    f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}_real')
        pred_image = cv2.imread(pred_file, cv2.IMREAD_GRAYSCALE)
        gt_image = cv2.imread(gt_file, cv2.IMREAD_GRAYSCALE)

        # SSIM
        ssim_score = ssim(pred_image, gt_image)
        ssim_emo = ssim_emo + ssim_score
        
        #PSRN
        psnr_score = calculate_psnr(pred_image, gt_image)
        psnr_emo = psnr_emo + psnr_score
        
        #CPBD
        cpbd_score = cpbd.compute(pred_image)
        cpbd_emo = cpbd_emo + cpbd_score
        
    # print(f"SSIM {i+1}: {ssim_emo/len(img_list)}")
    total_ssim.append(ssim_emo/len(img_list))
    total_psnr.append(psnr_emo/len(img_list))
    total_cpbd.append(cpbd_emo/len(img_list))

In [ ]:
print("Result:")
print(total_ssim)
print(f"Avg SSIM: {sum(total_ssim)/len(total_ssim)}")
print(total_psnr)
print(f"Avg PSNR: {sum(total_psnr)/len(total_psnr)}")

In [ ]:
print(total_cpbd)
print(f"Avg CPBD: {sum(total_cpbd)/len(total_cpbd)}")

In [ ]:
import lpips
import torch
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

total_lp = []
for i in range(6):
    lp_emo = 0
    img_list = sorted(glob(os.path.join(f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}', '*')))

    for pred_file in tqdm(img_list):
        gt_file = pred_file.replace(f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}',
                                    f'/root/TalkingHeadProject/imaginaire/logs/result_CREMAD/00{i+1}_real')
        pred_image = cv2.imread(pred_file)
        gt_image = cv2.imread(gt_file)
        
        pred_image = torch.from_numpy(np.transpose(pred_image, (2, 0, 1))).float() / 255.0
        gt_image = torch.from_numpy(np.transpose(gt_image, (2, 0, 1))).float() / 255.0

        lp_score = loss_fn_vgg(pred_image, gt_image)
        lp_emo = lp_emo + lp_score
    print(f"LPIPS {i+1}: {lp_emo/len(img_list)}")
    total_lp.append(lp_emo/len(img_list))
print(f"Avg LPIPS: {sum(total_lp)/len(total_lp)}")

In [ ]:
total_lp